In [1]:
import pandas as pd
import glob 

### Import and append all pressure csv files into one

In [2]:
# Path to the directory containing CSV files 
csv_files_path = r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Data\Pressure\*.csv" # Adjust this path as necessary 

# Use glob to get all CSV file paths 
csv_files = glob.glob(csv_files_path) 

# Use list comprehension to read all CSV files and concatenate them into a single DataFrame 
pressure_df = pd.concat([pd.read_csv(file,low_memory=False) for file in csv_files], ignore_index=True) 

# Now pressure_df contains all the data from the 20 CSV files 


In [3]:
#Drop unwanted columns
pressure_df.drop(['Parameter Code',
                  'Sample Duration', 
                  'Pollutant Standard', 
                  'Event Type', 
                  'Observation Count', 
                  'Observation Percent', 
                  'AQI', 
                  'Method Code', 
                  'Method Name', 
                  'Local Site Name', 
                  'Address', 
                  'Date of Last Change'],axis=1,inplace=True)

In [4]:
# Grouping by so we have 1 measurement per station per day
pressure_df = pressure_df.groupby(['State Code',
                                   'State Name',
                                   'County Code',
                                   'County Name',
                                   'City Name',
                                   'Site Num',
                                   'Datum',
                                   'Latitude',
                                   'Longitude',
                                   'Date Local',
                                   'Parameter Name',
                                   'Units of Measure']).agg({'Arithmetic Mean':'mean', '1st Max Value':'max','1st Max Hour':'max','CBSA Name':'first'}).reset_index()

In [5]:
#Drop Mexico, Virgin Islands and Puerto Rico
pressure_df.drop(pressure_df[(pressure_df['State Code']==80)|(pressure_df['State Code']==78)|(pressure_df['State Code']==72)].index,inplace=True)

In [6]:
#create unique identifier for each measurement
pressure_df['measurement_id'] = pressure_df['State Code'].apply(str)+'_'+pressure_df['County Code'].apply(str)+'_'+pressure_df['Site Num'].apply(str)+'_'+pressure_df['Date Local'].str.replace('-','')

In [7]:
#Renaming column to match the value measured and droping unnecessary columns
pressure_df.rename(columns={'Arithmetic Mean':'Barometric pressure(mb)','1st Max Value':'Barometric pressure(mb) Max Value', '1st Max Hour':'Barometric pressure(mb) Max Hour'},inplace=True)
pressure_df.drop(['Parameter Name', 'Units of Measure'],axis=1,inplace=True)

In [8]:
#Convert date column to datetime
pressure_df['Date Local'] = pd.to_datetime(pressure_df['Date Local'])

In [9]:
pressure_df.describe()

,State Code,County Code,Site Num,Latitude,Longitude,Date Local,Barometric pressure(mb),Barometric pressure(mb) Max Value,Barometric pressure(mb) Max Hour
count,1.949314e+06,1.949314e+06,1.949314e+06,1.949314e+06,1.949314e+06,1949314,1.949314e+06,1.949314e+06,1.949314e+06
mean,2.835530e+01,6.343131e+01,8.779120e+02,4.014453e+01,-9.772728e+01,2014-05-13 10:01:03.374909696,9.749914e+02,9.780129e+02,9.267589e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,1.942051e+01,-1.552879e+02,2004-01-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.600000e+01,1.900000e+01,6.000000e+00,3.719954e+01,-1.163479e+02,2010-01-24 00:00:00,9.139875e+02,9.170000e+02,0.000000e+00
50%,2.600000e+01,3.900000e+01,2.600000e+01,4.124749e+01,-9.529472e+01,2014-07-20 00:00:00,9.874657e+02,9.910000e+02,8.000000e+00
75%,4.100000e+01,8.300000e+01,1.005000e+03,4.286183e+01,-8.300014e+01,2018-11-10 00:00:00,1.004250e+03,1.007000e+03,1.900000e+01
max,5.600000e+01,5.100000e+02,9.997000e+03,6.484569e+01,-6.803301e+01,2023-09-30 00:00:00,3.515928e+04,3.518470e+04,2.300000e+01
std,1.697818e+01,7.266866e+01,1.741683e+03,4.416176e+00,1.778388e+01,NaN,7.844686e+02,7.871435e+02,8.135893e+00


In [10]:
#Clearing outliers

#Define reasonable ranges
reasonable_pressure_range = (870, 1100)  # Millibars

# Filter out unreasonable values and pressure
pressure_df.loc[~pressure_df['Barometric pressure(mb)'].between(*reasonable_pressure_range), 'Barometric pressure(mb)'] = None
pressure_df.loc[~pressure_df['Barometric pressure(mb) Max Value'].between(*reasonable_pressure_range), 'Barometric pressure(mb) Max Value'] = None

# Apply linear interpolation to fill NaN values
pressure_df[['Barometric pressure(mb)','Barometric pressure(mb) Max Value']].interpolate(method='linear', inplace=True)

C:\Users\stlva\AppData\Local\Temp\ipykernel_14008\1074877426.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pressure_df[['Barometric pressure(mb)','Barometric pressure(mb) Max Value']].interpolate(method='linear', inplace=True)


In [11]:
pressure_df.head()

,State Code,State Name,County Code,County Name,City Name,Site Num,Datum,Latitude,Longitude,Date Local,Barometric pressure(mb),Barometric pressure(mb) Max Value,Barometric pressure(mb) Max Hour,CBSA Name,measurement_id
0,1,Alabama,53,Escambia,Not in a city,1000,NAD83,31.0921,-87.5435,2016-01-01,1013.700000,1016.1,20,None,1_53_1000_20160101
1,1,Alabama,53,Escambia,Not in a city,1000,NAD83,31.0921,-87.5435,2016-01-02,1014.254167,1016.7,9,None,1_53_1000_20160102
2,1,Alabama,53,Escambia,Not in a city,1000,NAD83,31.0921,-87.5435,2016-01-03,1010.050000,1012.1,0,None,1_53_1000_20160103
3,1,Alabama,53,Escambia,Not in a city,1000,NAD83,31.0921,-87.5435,2016-01-04,1011.875000,1015.7,23,None,1_53_1000_20160104
4,1,Alabama,53,Escambia,Not in a city,1000,NAD83,31.0921,-87.5435,2016-01-05,1017.504167,1019.5,9,None,1_53_1000_20160105


In [12]:
#Saving final df to csv
#pressure_df.to_csv(r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Cleaning\all_pressure_clean.csv",index=False)
pressure_df.to_csv(r"C:\Users\stlva\Google Drive\Big blue data academy\main_branch\concatenated data\all_pressure_clean.csv",index=False)